In [ ]:
!pip install transformers
!pip install torch
!pip install pynvml
!pip install psutil
!pip install Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.6 MB/s eta 0:00:00


In [ ]:
# """
# Energy Consumption and Optimization of LLM Models in Training and Inference
# ------------------------------------------------------------------------

# Project Motivation:
# - Rising demand for LLM models and associated operational cost management.
# - Lack of energy efficiency insights and sustainability concerns.
# - Need for systematic measurement and comparison of energy usage and emissions.

# This project involves:
# 1. Experimental Setup:
#    - We will run experiments on Bridges2 or Google Colab (GPU environment) to measure:
#      - Inference energy/time/emissions for GPT-2 (LLM)
#      - Training energy/time/emissions for GPT-2 (small training iteration)
#      - Inference energy/time/emissions for an image recognition model (ResNet50)
#      - Inference energy/time/emissions for a sentence classification model (DistilBERT)

# 2. Model Selection and Experiments:
#    - GPT-2 for LLM inference and minimal training.
#    - ResNet50 (pretrained on ImageNet) for image recognition inference.
#    - DistilBERT (fine-tuned on SST-2) for sentence classification inference.

# 3. Data Collection and Metrics:
#    - Metrics: GPU power draw (Watts), CPU usage, runtime (s), estimated energy (Joules/Wh), CO2 emissions estimate.
#    - Tools: `pynvml` for GPU metrics, `psutil` for CPU metrics, timing for runtime.
#    - Energy calculation: Energy (Wh) = (Power (W) * Time (h))
#      Convert to Joules: 1 Wh = 3600 J.
#    - Emissions: Approximate using a carbon intensity factor (e.g., 0.4 kg CO2/kWh).
#      This is a rough approximation.

# 4. Data Analysis:
#    - Compare inference and training steps across models.
#    - Identify which model/process consumes more energy and produces more emissions.
#    - Propose optimization strategies.

# 5. Optimization Recommendations:
#    - Mixed precision inference.
#    - Model distillation.
#    - Efficient fine-tuning methods (e.g., LoRA, adapter training).
#    - Hardware-specific optimizations.

# Expected Results and Estimates:
# - LLM Inference (GPT-2): Fast, moderate energy, low emissions.
# - LLM Training: Energy-intensive, moderate emissions.
# - Image Recognition Inference (ResNet50): Quick, efficient, low emissions.
# - Sentence Classification Inference (DistilBERT): Fast, manageable energy and emissions.

# Note:
# - The code below is a representative template. Actual hardware, environment setup,
#   and credentials (like huggingface token if needed) must be provided.
# - Actual GPU/CPU metrics require a GPU environment with `nvidia-smi` or `pynvml` installed.
# - This code uses mock carbon intensity and approximations for demonstration purposes.

# Instructions:
# - Make sure `transformers`, `torch`, `pynvml`, `psutil`, `Pillow`, and `datasets` are installed.
# - Run on a GPU-enabled environment for accurate measurements.
# """

# import time
# import psutil
# import torch
# import torchvision.models as models
# import torchvision.transforms as transforms
# from PIL import Image
# import requests
# import io
# from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification
# from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
# import pynvml
# import csv
# import os

# ###############################
# # Configuration and Setup
# ###############################

# # Energy/Emissions constants (approximations)
# CARBON_INTENSITY = 0.4  # kg CO2 per kWh (example factor)
# # We'll record data to a CSV file
# RESULTS_CSV = "energy_measurements.csv"

# # Make sure we have GPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Initialize GPU metrics
# pynvml.nvmlInit()
# gpu_handle = pynvml.nvmlDeviceGetHandleByIndex(0)

# ###############################
# # Utility Functions
# ###############################

# def get_gpu_power_watts():
#     """Get current GPU power usage in watts."""
#     pinfo = pynvml.nvmlDeviceGetPowerUsage(gpu_handle)
#     # power usage in milliwatts, convert to watts
#     return pinfo / 1000.0

# def get_cpu_power_estimate():
#     """Estimate CPU power usage. This is a rough approximation using CPU utilization."""
#     # Without specialized instrumentation or RAPL, exact CPU power is hard to get.
#     # We'll just record CPU utilization as a proxy.
#     cpu_util = psutil.cpu_percent()
#     # A rough estimate: Suppose full CPU usage ~50W for the system.
#     # This is a guess and should be replaced with real hardware-specific measurement.
#     estimated_cpu_power = (cpu_util / 100.0) * 50
#     return estimated_cpu_power

# def measure_energy_and_time(func, *args, **kwargs):
#     """
#     Measure the runtime and approximate energy usage for a given function.
#     We'll measure GPU and CPU power multiple times and integrate over the runtime.
#     """
#     start_time = time.time()
#     energy_gpu_joules = 0.0
#     energy_cpu_joules = 0.0

#     # We'll sample power usage at intervals
#     interval = 0.1  # 100ms sampling
#     running = True
#     import threading

#     def target():
#         func(*args, **kwargs)

#     thread = threading.Thread(target=target)
#     thread.start()

#     while thread.is_alive():
#         gpu_watts = get_gpu_power_watts()
#         cpu_watts = get_cpu_power_estimate()
#         time.sleep(interval)
#         energy_gpu_joules += gpu_watts * (interval) * 1.0  # W * s = J
#         energy_cpu_joules += cpu_watts * (interval) * 1.0  # W * s = J

#     thread.join()
#     end_time = time.time()
#     total_time = end_time - start_time
#     total_energy_joules = energy_gpu_joules + energy_cpu_joules

#     # Convert joules to kWh: 1 J = 2.77778e-7 kWh
#     total_energy_kwh = total_energy_joules * 2.77778e-7

#     # Emissions:
#     # Emissions (kg CO2) = total_energy_kwh * CARBON_INTENSITY
#     emissions_kg = total_energy_kwh * CARBON_INTENSITY

#     return total_time, total_energy_joules, emissions_kg

# def log_results(task_name, model_name, phase, total_time, energy_joules, emissions_kg):
#     file_exists = os.path.isfile(RESULTS_CSV)
#     with open(RESULTS_CSV, 'a', newline='') as f:
#         writer = csv.writer(f)
#         if not file_exists:
#             writer.writerow(["Task", "Model", "Phase", "Time(s)", "Energy(J)", "Emissions(kgCO2)"])
#         writer.writerow([task_name, model_name, phase, total_time, energy_joules, emissions_kg])

# ###############################
# # Models and Experiments Setup
# ###############################

# # 1. LLM: GPT-2 Inference and minimal training demonstration
# llm_tokenizer = AutoTokenizer.from_pretrained("gpt2")
# llm_model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)
# llm_model.eval()

# # 2. Image Recognition: ResNet50
# resnet_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2).to(device)
# resnet_model.eval()
# img_transform = transforms.Compose([
#     transforms.Resize((224,224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485,0.456,0.406],
#                          std=[0.229,0.224,0.225]),
# ])

# # 3. Sentence Classification: DistilBERT (SST-2)
# distil_bert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# distil_bert_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english").to(device)
# distil_bert_model.eval()

# ###############################
# # Example Inputs
# ###############################

# # LLM inference input
# llm_input_text = "The future of energy-efficient AI models is"
# llm_inputs = llm_tokenizer(llm_input_text, return_tensors='pt').to(device)

# def llm_inference():
#     with torch.no_grad():
#         outputs = llm_model.generate(**llm_inputs, max_length=50)

# # LLM training setup (small dummy training step)
# # We'll create a tiny random dataset of next-token prediction for demonstration.
# def llm_training_step():
#     llm_model.train()
#     optimizer = torch.optim.AdamW(llm_model.parameters(), lr=1e-5)
#     dummy_input_ids = torch.randint(0, llm_model.config.vocab_size, (2, 16)).to(device) # batch 2, seq 16
#     dummy_labels = dummy_input_ids.clone()
#     outputs = llm_model(dummy_input_ids, labels=dummy_labels)
#     loss = outputs.loss
#     loss.backward()
#     optimizer.step()

# # Image Recognition Inference input
# img_pil = Image.open("dog.jpg")
# img_tensor = img_transform(img_pil).unsqueeze(0).to(device)

# def image_inference():
#     with torch.no_grad():
#         res = resnet_model(img_tensor)
#         res = resnet_model(img_tensor)

# # Sentence Classification Inference input
# sent_input_text = "I love this movie!"
# sent_inputs = distil_bert_tokenizer(sent_input_text, return_tensors="pt", truncation=True, padding=True).to(device)

# def sentence_inference():
#     with torch.no_grad():
#         out = distil_bert_model(**sent_inputs)

# ###############################
# # Run Experiments and Record
# ###############################

# # LLM Inference
# time_, energy_j, emissions_kg = measure_energy_and_time(llm_inference)
# log_results("LLM Inference", "GPT-2", "Inference", time_, energy_j, emissions_kg)

# # LLM Training (just a single step demo, in practice you'd loop multiple steps)
# time_, energy_j, emissions_kg = measure_energy_and_time(llm_training_step)
# log_results("LLM Training", "GPT-2", "Training", time_, energy_j, emissions_kg)

# # Image Recognition Inference
# time_, energy_j, emissions_kg = measure_energy_and_time(image_inference)
# log_results("Image Recognition Inference", "ResNet50", "Inference", time_, energy_j, emissions_kg)

# # Sentence Classification Inference
# time_, energy_j, emissions_kg = measure_energy_and_time(sentence_inference)
# log_results("Sentence Classification Inference", "DistilBERT", "Inference", time_, energy_j, emissions_kg)

# ###############################
# # Optimization Example (Optional)
# ###############################

# # For example, try mixed-precision inference for GPT-2
# # This will reduce energy/time slightly, we can measure again
# llm_model_fp16 = AutoModelForCausalLM.from_pretrained("gpt2", torch_dtype=torch.float16).to(device)
# llm_model_fp16.eval()

# def llm_inference_fp16():
#     with torch.no_grad():
#         outputs = llm_model_fp16.generate(**llm_inputs, max_length=50)

# time_, energy_j, emissions_kg = measure_energy_and_time(llm_inference_fp16)
# log_results("LLM Inference Optimized", "GPT-2 FP16", "Inference", time_, energy_j, emissions_kg)

# ###############################
# # Further steps (not coded):
# # - Analyze results in CSV
# # - Plot energy/time tradeoffs
# # - Provide recommendations
# ###############################

# print("Experiments completed. Results logged to:", RESULTS_CSV)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Experiments completed. Results logged to: energy_measurements.csv


In [ ]:
# # -*- coding: utf-8 -*-
# """GPT2_Transfer Learning with Energy Measurement

# This code:
# - Performs text classification on tweets (sentiment analysis) using GPT-2 as a feature extractor.
# - Uses a subset of the data for quicker experimentation.
# - Measures energy consumption (GPU & CPU) and execution time for training and evaluation steps.
# - Logs results (time, energy, emissions) into a CSV file.

# """

# # !pip install transformers psutil pynvml

# import math
# import tensorflow as tf
# from transformers import GPT2Tokenizer, TFGPT2Model
# from google.colab import drive
# import pandas as pd

# import time
# import psutil
# import torch
# import pynvml
# import threading
# import csv
# import os

# ###############################
# # Mount Google Drive
# ###############################
# # drive.mount('/content/gdrive')

# ###############################
# # Energy/Emissions constants (approximations)
# ###############################
# CARBON_INTENSITY = 0.4  # kg CO2 per kWh (example factor)
# RESULTS_CSV1 = "energy_measurements1.csv"

# ###############################
# # GPU Initialization for Energy Measurement
# ###############################
# pynvml.nvmlInit()
# gpu_handle = pynvml.nvmlDeviceGetHandleByIndex(0)

# def get_gpu_power_watts():
#     """Get current GPU power usage in watts."""
#     pinfo = pynvml.nvmlDeviceGetPowerUsage(gpu_handle)
#     return pinfo / 1000.0

# def get_cpu_power_estimate():
#     """
#     Estimate CPU power usage.
#     This is a rough approximation.
#     Adjust based on your hardware or use RAPL if available.
#     """
#     cpu_util = psutil.cpu_percent()
#     # Assume full CPU usage ~50W for the system
#     estimated_cpu_power = (cpu_util / 100.0) * 50
#     return estimated_cpu_power

# def measure_energy_and_time(func, *args, **kwargs):
#     """
#     Measure runtime and approximate energy usage for a given function call.
#     Sampling GPU/CPU usage every 100ms.
#     """
#     start_time = time.time()
#     energy_gpu_joules = 0.0
#     energy_cpu_joules = 0.0
#     interval = 0.1

#     def target():
#         func(*args, **kwargs)

#     thread = threading.Thread(target=target)
#     thread.start()

#     while thread.is_alive():
#         gpu_watts = get_gpu_power_watts()
#         cpu_watts = get_cpu_power_estimate()
#         time.sleep(interval)
#         energy_gpu_joules += gpu_watts * interval
#         energy_cpu_joules += cpu_watts * interval

#     thread.join()
#     end_time = time.time()
#     total_time = end_time - start_time
#     total_energy_joules = energy_gpu_joules + energy_cpu_joules
#     total_energy_kwh = total_energy_joules * 2.77778e-7
#     emissions_kg = total_energy_kwh * CARBON_INTENSITY

#     return total_time, total_energy_joules, emissions_kg

# def log_results(task_name, phase, total_time, energy_joules, emissions_kg, filename=RESULTS_CSV):
#     file_exists = os.path.isfile(filename)
#     with open(filename, 'a', newline='') as f:
#         writer = csv.writer(f)
#         if not file_exists:
#             writer.writerow(["Task", "Phase", "Time(s)", "Energy(J)", "Emissions(kgCO2)"])
#         writer.writerow([task_name, phase, total_time, energy_joules, emissions_kg])

# ###############################
# # Load Data
# ###############################
# # Ensure that your data file is accessible in your drive
# # Adjust the path if needed.
# df = pd.read_csv('tweets_processed.csv')

# # Use a subset of data for quicker tests (e.g., 10% of original data)
# df = df.sample(frac=0.1, random_state=42).reset_index(drop=True)

# from sklearn.model_selection import train_test_split

# X = df['lemmatized_tweets']
# y = df['VADER_sentiment']

# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# ###############################
# # Preprocessing and Tokenization
# ###############################
# MAX_LENGTH = math.ceil((X_train.apply(lambda x: len(str(x).split())).mean()))+2

# PAD_TOKEN = "<|pad|>"
# EOS_TOKEN = "<|endoftext|>"

# tokenizer = GPT2Tokenizer.from_pretrained("gpt2",
#     pad_token=PAD_TOKEN,
#     eos_token=EOS_TOKEN,
#     max_length=MAX_LENGTH,
#     is_split_into_words=True)

# X_train = [str(ex) + EOS_TOKEN for ex in X_train]
# X_test = [str(ex) + EOS_TOKEN for ex in X_test]

# X_train_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
#                       truncation=True, padding="max_length", add_special_tokens=True)['input_ids'] for x in X_train]
# X_test_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
#                      truncation=True, padding="max_length", add_special_tokens=True)['input_ids'] for x in X_test]

# X_train_in = tf.squeeze(tf.convert_to_tensor(X_train_), axis=1)
# X_test_in = tf.squeeze(tf.convert_to_tensor(X_test_), axis=1)

# X_train_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
#                            truncation=True, padding="max_length", add_special_tokens=True)["attention_mask"] for x in X_train]
# X_test_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
#                           truncation=True, padding="max_length", add_special_tokens=True)["attention_mask"] for x in X_test]

# X_train_mask = tf.squeeze(tf.convert_to_tensor(X_train_mask_), axis=1)
# X_test_mask = tf.squeeze(tf.convert_to_tensor(X_test_mask_), axis=1)

# ###############################
# # Model Setup
# ###############################
# model = TFGPT2Model.from_pretrained("gpt2", use_cache=False,
#         pad_token_id=tokenizer.pad_token_id,
#         eos_token_id=tokenizer.eos_token_id)
# model.training = True
# model.resize_token_embeddings(len(tokenizer))

# # Freeze GPT2 layers
# for layer in model.layers:
#     layer.trainable = False

# input_ = tf.keras.layers.Input(shape=(None,), dtype='int32')
# mask_ = tf.keras.layers.Input(shape=(None,), dtype='int32')
# x = model(input_, attention_mask=mask_)
# x = tf.reduce_mean(x.last_hidden_state, axis=1)
# x = tf.keras.layers.Dense(16, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.3)(x)
# output = tf.keras.layers.Dense(3, activation='softmax')(x)

# clf = tf.keras.Model([input_, mask_], output)

# base_learning_rate = 0.0005
# optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
# loss = tf.keras.losses.SparseCategoricalCrossentropy()

# clf.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# callbacks = tf.keras.callbacks.EarlyStopping(
#     monitor="accuracy", verbose=1, patience=3, restore_best_weights=True)

# def map_sentiment(value):
#   if value == 'Negative':
#     return 0
#   if value == 'Neutral':
#     return 1
#   if value == 'Positive':
#     return 2

# y_train_ = y_train.map(map_sentiment)
# y_test_ = y_test.map(map_sentiment)

# y_train_in = tf.constant(y_train_, dtype=tf.int32)
# y_test_in = tf.constant(y_test_, dtype=tf.int32)

# tf.config.experimental_run_functions_eagerly(True)

# def train_model():
#     clf.fit([X_train_in, X_train_mask], y_train_in, epochs=5, batch_size=32, validation_split=0.2, callbacks=callbacks)

# ###############################
# # Training with Energy Measurement
# ###############################
# print("Starting training...")
# train_time, train_energy_j, train_emissions_kg = measure_energy_and_time(train_model)
# print(f"Training completed in {train_time:.2f}s, Energy: {train_energy_j:.2f}J, Emissions: {train_emissions_kg:.4f}kg CO2")
# log_results("GPT2_TransferLearning", "Training", train_time, train_energy_j, train_emissions_kg)

# ###############################
# # Evaluation with Energy Measurement
# ###############################
# print("Evaluating on test set...")
# eval_time, eval_energy_j, eval_emissions_kg = measure_energy_and_time(clf.evaluate, [X_test_in, X_test_mask], y_test_in)
# print(f"Evaluation completed in {eval_time:.2f}s, Energy: {eval_energy_j:.2f}J, Emissions: {eval_emissions_kg:.4f}kg CO2")
# log_results("GPT2_TransferLearning", "Evaluation", eval_time, eval_energy_j, eval_emissions_kg)

# clf.training = False
# y_pred = clf.predict([X_test_in, X_test_mask])

# print("Done. Results saved to:", RESULTS_CSV1)


All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


Starting training...
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


4/4 [==============================] - 3s 925ms/step - loss: 11.2003 - accuracy: 0.4113 - val_loss: 9.7950 - val_accuracy: 0.2500
Epoch 2/5
4/4 [==============================] - 2s 612ms/step - loss: 11.0644 - accuracy: 0.3952 - val_loss: 8.1811 - val_accuracy: 0.2500
Epoch 3/5
4/4 [==============================] - 2s 610ms/step - loss: 11.1697 - accuracy: 0.3306 - val_loss: 6.9311 - val_accuracy: 0.2500
Epoch 4/5
4/4 [==============================] - 3s 801ms/step - loss: 7.8770 - accuracy: 0.4839 - val_loss: 5.6655 - val_accuracy: 0.2500
Epoch 5/5
4/4 [==============================] - 3s 752ms/step - loss: 8.5798 - accuracy: 0.4032 - val_loss: 4.4446 - val_accuracy: 0.2812
Restoring model weights from the end of the best epoch: 4.
Training completed in 20.63s, Energy: 1235.18J, Emissions: 0.0001kg CO2
Evaluating on test set...
2/2 [==============================] - 0s 208ms/step - loss: 4.8946 - accuracy: 0.3585
Evaluation completed in 0.52s, Energy: 32.15J, Emissions: 0.0000kg C

In [ ]:
# -*- coding: utf-8 -*-
"""GPT2_Transfer Learning with Energy Measurement"""

# Install necessary libraries
!pip install transformers psutil pynvml

# Import required libraries
import math
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2Model
from google.colab import drive
import pandas as pd
import subprocess
import time
import psutil
import threading
import csv
import os

###############################
# Constants and Setup
###############################
CARBON_INTENSITY = 0.4  # kg CO2 per kWh (example factor)
RESULTS_CSV = "energy_measurements_gpu.csv"
USE_GPU = tf.config.list_physical_devices('GPU')  # Check if GPU is available

###############################
# Mount Google Drive
###############################
# drive.mount('/content/gdrive')

###############################
# GPU and CPU Power Functions
###############################

# Function to Measure Power Usage with nvidia-smi
def get_gpu_power():
    if USE_GPU:
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=power.draw", "--format=csv,nounits,noheader"],
            stdout=subprocess.PIPE, text=True)
        return float(result.stdout.strip())
    return 0.0

# Estimate CPU power usage
def get_cpu_power_estimate():
    """
    Estimate CPU power usage.
    This is a rough approximation.
    """
    cpu_util = psutil.cpu_percent()
    estimated_cpu_power = (cpu_util / 100.0) * 50
    return estimated_cpu_power

# Measure energy consumption and time
def measure_energy_and_time(func, *args, **kwargs):
    """
    Measure runtime and approximate energy usage for a given function call.
    This now includes an alternative method using `nvidia-smi` for GPU power monitoring.
    """
    start_time = time.time()
    energy_gpu_joules = 0.0
    energy_cpu_joules = 0.0
    interval = 0.1  # Sampling interval in seconds

    def target():
        func(*args, **kwargs)

    thread = threading.Thread(target=target)
    thread.start()

    while thread.is_alive():
        # Measure GPU power using nvidia-smi or pynvml
        gpu_watts = get_gpu_power()
        cpu_watts = get_cpu_power_estimate()
        time.sleep(interval)
        energy_gpu_joules += gpu_watts * interval
        energy_cpu_joules += cpu_watts * interval

    thread.join()
    end_time = time.time()
    total_time = end_time - start_time
    total_energy_joules = energy_gpu_joules + energy_cpu_joules
    total_energy_kwh = total_energy_joules * 2.77778e-7
    emissions_kg = total_energy_kwh * CARBON_INTENSITY

    return total_time, total_energy_joules, emissions_kg

# Logging function
def log_results(task_name, phase, total_time, energy_joules, emissions_kg, filename=RESULTS_CSV):
    file_exists = os.path.isfile(filename)
    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["Task", "Phase", "Time(s)", "Energy(J)", "Emissions(kgCO2)"])
        writer.writerow([task_name, phase, total_time, energy_joules, emissions_kg])

###############################
# Load Data
###############################
# Ensure that your data file is accessible in your drive
df = pd.read_csv('tweets_processed.csv')

# Use a subset of data for quicker tests (e.g., 10% of original data)
df = df.sample(frac=0.1, random_state=42).reset_index(drop=True)

from sklearn.model_selection import train_test_split

X = df['lemmatized_tweets']
y = df['VADER_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

###############################
# Preprocessing and Tokenization
###############################
MAX_LENGTH = math.ceil((X_train.apply(lambda x: len(str(x).split())).mean()))+2

PAD_TOKEN = "<|pad|>"
EOS_TOKEN = "<|endoftext|>"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2",
    pad_token=PAD_TOKEN,
    eos_token=EOS_TOKEN,
    max_length=MAX_LENGTH,
    is_split_into_words=True)

X_train = [str(ex) + EOS_TOKEN for ex in X_train]
X_test = [str(ex) + EOS_TOKEN for ex in X_test]

X_train_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                      truncation=True, padding="max_length", add_special_tokens=True)['input_ids'] for x in X_train]
X_test_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                     truncation=True, padding="max_length", add_special_tokens=True)['input_ids'] for x in X_test]

X_train_in = tf.squeeze(tf.convert_to_tensor(X_train_), axis=1)
X_test_in = tf.squeeze(tf.convert_to_tensor(X_test_), axis=1)

X_train_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                           truncation=True, padding="max_length", add_special_tokens=True)["attention_mask"] for x in X_train]
X_test_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                          truncation=True, padding="max_length", add_special_tokens=True)["attention_mask"] for x in X_test]

X_train_mask = tf.squeeze(tf.convert_to_tensor(X_train_mask_), axis=1)
X_test_mask = tf.squeeze(tf.convert_to_tensor(X_test_mask_), axis=1)

###############################
# Model Setup
###############################
model = TFGPT2Model.from_pretrained("gpt2", use_cache=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id)
model.training = True
model.resize_token_embeddings(len(tokenizer))

# Freeze GPT2 layers
for layer in model.layers:
    layer.trainable = False

input_ = tf.keras.layers.Input(shape=(None,), dtype='int32')
mask_ = tf.keras.layers.Input(shape=(None,), dtype='int32')
x = model(input_, attention_mask=mask_)
x = tf.reduce_mean(x.last_hidden_state, axis=1)
x = tf.keras.layers.Dense(16, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(3, activation='softmax')(x)

clf = tf.keras.Model([input_, mask_], output)

base_learning_rate = 0.0005
optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

clf.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

callbacks = tf.keras.callbacks.EarlyStopping(
    monitor="accuracy", verbose=1, patience=3, restore_best_weights=True)

def map_sentiment(value):
  if value == 'Negative':
    return 0
  if value == 'Neutral':
    return 1
  if value == 'Positive':
    return 2

y_train_ = y_train.map(map_sentiment)
y_test_ = y_test.map(map_sentiment)

y_train_in = tf.constant(y_train_, dtype=tf.int32)
y_test_in = tf.constant(y_test_, dtype=tf.int32)

tf.config.experimental_run_functions_eagerly(True)

def train_model():
    clf.fit([X_train_in, X_train_mask], y_train_in, epochs=5, batch_size=32, validation_split=0.2, callbacks=callbacks)

###############################
# Training with Energy Measurement
###############################
print("Starting training...")
train_time, train_energy_j, train_emissions_kg = measure_energy_and_time(train_model)
print(f"Training completed in {train_time:.2f}s, Energy: {train_energy_j:.2f}J, Emissions: {train_emissions_kg:.4f}kg CO2")
log_results("GPT2_TransferLearning", "Training", train_time, train_energy_j, train_emissions_kg)

###############################
# Evaluation with Energy Measurement
###############################
print("Evaluating on test set...")
eval_time, eval_energy_j, eval_emissions_kg = measure_energy_and_time(clf.evaluate, [X_test_in, X_test_mask], y_test_in)
print(f"Evaluation completed in {eval_time:.2f}s, Energy: {eval_energy_j:.2f}J, Emissions: {eval_emissions_kg:.4f}kg CO2")
log_results("GPT2_TransferLearning", "Evaluation", eval_time, eval_energy_j, eval_emissions_kg)

clf.training = False
y_pred = clf.predict([X_test_in, X_test_mask])

print("Done. Results saved to:", RESULTS_CSV)


All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


Starting training...
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


4/4 [==============================] - ETA: 0s - loss: 6.8715 - accuracy: 0.2984

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


4/4 [==============================] - 4s 993ms/step - loss: 6.8715 - accuracy: 0.2984 - val_loss: 8.1579 - val_accuracy: 0.1250
Epoch 2/5
4/4 [==============================] - 3s 775ms/step - loss: 6.4132 - accuracy: 0.3065 - val_loss: 6.4392 - val_accuracy: 0.1250
Epoch 3/5
4/4 [==============================] - 4s 983ms/step - loss: 5.2451 - accuracy: 0.3145 - val_loss: 4.8973 - val_accuracy: 0.1250
Epoch 4/5
4/4 [==============================] - 3s 750ms/step - loss: 4.5685 - accuracy: 0.3468 - val_loss: 3.6648 - val_accuracy: 0.1250
Epoch 5/5
4/4 [==============================] - 2s 624ms/step - loss: 3.8470 - accuracy: 0.3468 - val_loss: 2.8701 - val_accuracy: 0.2500
Restoring model weights from the end of the best epoch: 4.
Training completed in 20.58s, Energy: 1066.56J, Emissions: 0.0001kg CO2
Evaluating on test set...


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


2/2 [==============================] - 1s 344ms/step - loss: 3.0356 - accuracy: 0.2830
Evaluation completed in 0.81s, Energy: 49.05J, Emissions: 0.0000kg CO2


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


2/2 [==============================] - 0s 213ms/step
Done. Results saved to: energy_measurements.csv


### Remove Frezzing

In [ ]:
# -*- coding: utf-8 -*-
"""GPT2_Transfer Learning with Energy Measurement and GPU Insights"""

# Install necessary libraries
!pip install transformers psutil pynvml

# Import required libraries
import math
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2Model
from google.colab import drive
import pandas as pd
import subprocess
import time
import psutil
import threading
import csv
import os
import pynvml

###############################
# Constants and Setup
###############################
CARBON_INTENSITY = 0.4  # kg CO2 per kWh (example factor)
RESULTS_CSV = "energy_measurements_freze_gpu.csv"
GPU_DETAILS_CSV = "gpu_details_freze_gpu.csv"
USE_GPU = tf.config.list_physical_devices('GPU')  # Check if GPU is available

###############################
# GPU and CPU Power Functions
###############################

def get_gpu_details():
    """
    Retrieve and print GPU architecture and other insights using NVML.
    """
    pynvml.nvmlInit()
    device_count = pynvml.nvmlDeviceGetCount()
    details = []

    for i in range(device_count):
        handle = pynvml.nvmlDeviceGetHandleByIndex(i)
        name = pynvml.nvmlDeviceGetName(handle)  # No need to decode
        memory_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        utilization = pynvml.nvmlDeviceGetUtilizationRates(handle)
        architecture = pynvml.nvmlSystemGetCudaDriverVersion()

        details.append({
            "GPU": name,
            "Memory Total (MB)": memory_info.total // (1024 ** 2),
            "Memory Free (MB)": memory_info.free // (1024 ** 2),
            "Memory Used (MB)": memory_info.used // (1024 ** 2),
            "GPU Utilization (%)": utilization.gpu,
            "Memory Utilization (%)": utilization.memory,
            "CUDA Version": architecture
        })

    pynvml.nvmlShutdown()
    return details


# Function to Measure Power Usage with nvidia-smi
def get_gpu_power():
    if USE_GPU:
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=power.draw", "--format=csv,nounits,noheader"],
            stdout=subprocess.PIPE, text=True)
        return float(result.stdout.strip())
    return 0.0

# Estimate CPU power usage
def get_cpu_power_estimate():
    """
    Estimate CPU power usage.
    This is a rough approximation.
    """
    cpu_util = psutil.cpu_percent()
    estimated_cpu_power = (cpu_util / 100.0) * 50
    return estimated_cpu_power

# Measure energy consumption and time
def measure_energy_and_time(func, *args, **kwargs):
    """
    Measure runtime and approximate energy usage for a given function call.
    """
    start_time = time.time()
    energy_gpu_joules = 0.0
    energy_cpu_joules = 0.0
    interval = 0.1  # Sampling interval in seconds

    def target():
        func(*args, **kwargs)

    thread = threading.Thread(target=target)
    thread.start()

    while thread.is_alive():
        # Measure GPU power using nvidia-smi
        gpu_watts = get_gpu_power()
        cpu_watts = get_cpu_power_estimate()
        time.sleep(interval)
        energy_gpu_joules += gpu_watts * interval
        energy_cpu_joules += cpu_watts * interval

    thread.join()
    end_time = time.time()
    total_time = end_time - start_time
    total_energy_joules = energy_gpu_joules + energy_cpu_joules
    total_energy_kwh = total_energy_joules * 2.77778e-7
    emissions_kg = total_energy_kwh * CARBON_INTENSITY

    return total_time, total_energy_joules, emissions_kg

# Logging functions
def log_results(task_name, phase, total_time, energy_joules, emissions_kg, filename=RESULTS_CSV):
    file_exists = os.path.isfile(filename)
    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["Task", "Phase", "Time(s)", "Energy(J)", "Emissions(kgCO2)"])
        writer.writerow([task_name, phase, total_time, energy_joules, emissions_kg])

def log_gpu_details(details, filename=GPU_DETAILS_CSV):
    file_exists = os.path.isfile(filename)
    with open(filename, 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=details[0].keys())
        if not file_exists:
            writer.writeheader()
        writer.writerows(details)

###############################
# Load Data
###############################
df = pd.read_csv('tweets_processed.csv')

# Use a subset of data for quicker tests (e.g., 10% of original data)
df = df.sample(frac=0.6, random_state=42).reset_index(drop=True)

from sklearn.model_selection import train_test_split

X = df['lemmatized_tweets']
y = df['VADER_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

###############################
# Preprocessing and Tokenization
###############################
MAX_LENGTH = math.ceil((X_train.apply(lambda x: len(str(x).split())).mean()))+2

PAD_TOKEN = "<|pad|>"
EOS_TOKEN = "<|endoftext|>"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2",
    pad_token=PAD_TOKEN,
    eos_token=EOS_TOKEN,
    max_length=MAX_LENGTH,
    is_split_into_words=True)

X_train = [str(ex) + EOS_TOKEN for ex in X_train]
X_test = [str(ex) + EOS_TOKEN for ex in X_test]

X_train_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                      truncation=True, padding="max_length", add_special_tokens=True)['input_ids'] for x in X_train]
X_test_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                     truncation=True, padding="max_length", add_special_tokens=True)['input_ids'] for x in X_test]

X_train_in = tf.squeeze(tf.convert_to_tensor(X_train_), axis=1)
X_test_in = tf.squeeze(tf.convert_to_tensor(X_test_), axis=1)

X_train_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                           truncation=True, padding="max_length", add_special_tokens=True)["attention_mask"] for x in X_train]
X_test_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                          truncation=True, padding="max_length", add_special_tokens=True)["attention_mask"] for x in X_test]

X_train_mask = tf.squeeze(tf.convert_to_tensor(X_train_mask_), axis=1)
X_test_mask = tf.squeeze(tf.convert_to_tensor(X_test_mask_), axis=1)

###############################
# Model Setup
###############################
model = TFGPT2Model.from_pretrained("gpt2", use_cache=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id)
model.training = True
model.resize_token_embeddings(len(tokenizer))

input_ = tf.keras.layers.Input(shape=(None,), dtype='int32')
mask_ = tf.keras.layers.Input(shape=(None,), dtype='int32')
x = model(input_, attention_mask=mask_)
x = tf.reduce_mean(x.last_hidden_state, axis=1)
x = tf.keras.layers.Dense(16, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(3, activation='softmax')(x)

clf = tf.keras.Model([input_, mask_], output)

base_learning_rate = 0.0005
optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

clf.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

callbacks = tf.keras.callbacks.EarlyStopping(
    monitor="accuracy", verbose=1, patience=3, restore_best_weights=True)

def map_sentiment(value):
  if value == 'Negative':
    return 0
  if value == 'Neutral':
    return 1
  if value == 'Positive':
    return 2

y_train_ = y_train.map(map_sentiment)
y_test_ = y_test.map(map_sentiment)

y_train_in = tf.constant(y_train_, dtype=tf.int32)
y_test_in = tf.constant(y_test_, dtype=tf.int32)

tf.config.experimental_run_functions_eagerly(True)

def train_model():
    clf.fit([X_train_in, X_train_mask], y_train_in, epochs=30, batch_size=32, validation_split=0.2, callbacks=callbacks)

###############################
# Gather GPU Details
###############################
gpu_details = get_gpu_details()
print("GPU Details and Insights:")
for detail in gpu_details:
    print(detail)
log_gpu_details(gpu_details)

###############################
# Training with Energy Measurement
###############################
print("Starting training...")
train_time, train_energy_j, train_emissions_kg = measure_energy_and_time(train_model)
print(f"Training completed in {train_time:.2f}s, Energy: {train_energy_j:.2f}J, Emissions: {train_emissions_kg:.4f}kg CO2")
log_results("GPT2_TransferLearning", "Training", train_time, train_energy_j, train_emissions_kg)

###############################
# Evaluation with Energy Measurement
###############################
print("Evaluating on test set...")
eval_time, eval_energy_j, eval_emissions_kg = measure_energy_and_time(clf.evaluate, [X_test_in, X_test_mask], y_test_in)
print(f"Evaluation completed in {eval_time:.2f}s, Energy: {eval_energy_j:.2f}J, Emissions: {eval_emissions_kg:.4f}kg CO2")
log_results("GPT2_TransferLearning", "Evaluation", eval_time, eval_energy_j, eval_emissions_kg)

clf.training = False
y_pred = clf.predict([X_test_in, X_test_mask])

print("Done. Results saved to:", RESULTS_CSV)


All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


GPU Details and Insights:
{'GPU': 'Tesla T4', 'Memory Total (MB)': 15360, 'Memory Free (MB)': 6793, 'Memory Used (MB)': 8566, 'GPU Utilization (%)': 0, 'Memory Utilization (%)': 0, 'CUDA Version': 12020}
Starting training...


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/30
24/24 [==============================] - ETA: 0s - loss: 1.8019 - accuracy: 0.3533

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


24/24 [==============================] - 27s 1s/step - loss: 1.8019 - accuracy: 0.3533 - val_loss: 1.0890 - val_accuracy: 0.4392
Epoch 2/30
24/24 [==============================] - 25s 1s/step - loss: 1.1001 - accuracy: 0.3732 - val_loss: 1.0596 - val_accuracy: 0.4603
Epoch 3/30
24/24 [==============================] - 25s 1s/step - loss: 1.0729 - accuracy: 0.4170 - val_loss: 1.0744 - val_accuracy: 0.3228
Epoch 4/30
24/24 [==============================] - 25s 1s/step - loss: 1.0762 - accuracy: 0.3559 - val_loss: 1.0937 - val_accuracy: 0.3228
Epoch 5/30
24/24 [==============================] - 25s 1s/step - loss: 1.0842 - accuracy: 0.3679 - val_loss: 1.0790 - val_accuracy: 0.3228
Epoch 6/30
24/24 [==============================] - 26s 1s/step - loss: 1.0688 - accuracy: 0.3665 - val_loss: 1.0775 - val_accuracy: 0.4603
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 3.
Training completed in 154.58s, Energy: 9198.47J, Emissions: 0.0010kg CO2
Evaluating on t

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


10/10 [==============================] - 2s 224ms/step - loss: 1.0654 - accuracy: 0.3599
Evaluation completed in 50.29s, Energy: 1819.57J, Emissions: 0.0002kg CO2


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


10/10 [==============================] - 2s 200ms/step
Done. Results saved to: energy_measurements_freze_gpu.csv


In [ ]:
# -*- coding: utf-8 -*-
"""GPT2_Transfer Learning with Energy Measurement and GPU Insights"""

# Install necessary libraries
!pip install transformers psutil pynvml

# Import required libraries
import math
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2Model
from google.colab import drive
import pandas as pd
import subprocess
import time
import psutil
import threading
import csv
import os
import pynvml

###############################
# Constants and Setup
###############################
CARBON_INTENSITY = 0.4  # kg CO2 per kWh (example factor)
RESULTS_CSV = "energy_measurements_gpu.csv"
GPU_DETAILS_CSV = "gpu_details_gpu.csv"
USE_GPU = tf.config.list_physical_devices('GPU')  # Check if GPU is available

###############################
# GPU and CPU Power Functions
###############################

def get_gpu_details():
    """
    Retrieve and print GPU architecture and other insights using NVML.
    """
    pynvml.nvmlInit()
    device_count = pynvml.nvmlDeviceGetCount()
    details = []

    for i in range(device_count):
        handle = pynvml.nvmlDeviceGetHandleByIndex(i)
        name = pynvml.nvmlDeviceGetName(handle)  # No need to decode
        memory_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        utilization = pynvml.nvmlDeviceGetUtilizationRates(handle)
        architecture = pynvml.nvmlSystemGetCudaDriverVersion()

        details.append({
            "GPU": name,
            "Memory Total (MB)": memory_info.total // (1024 ** 2),
            "Memory Free (MB)": memory_info.free // (1024 ** 2),
            "Memory Used (MB)": memory_info.used // (1024 ** 2),
            "GPU Utilization (%)": utilization.gpu,
            "Memory Utilization (%)": utilization.memory,
            "CUDA Version": architecture
        })

    pynvml.nvmlShutdown()
    return details


# Function to Measure Power Usage with nvidia-smi
def get_gpu_power():
    if USE_GPU:
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=power.draw", "--format=csv,nounits,noheader"],
            stdout=subprocess.PIPE, text=True)
        return float(result.stdout.strip())
    return 0.0

# Estimate CPU power usage
def get_cpu_power_estimate():
    """
    Estimate CPU power usage.
    This is a rough approximation.
    """
    cpu_util = psutil.cpu_percent()
    estimated_cpu_power = (cpu_util / 100.0) * 50
    return estimated_cpu_power

# Measure energy consumption and time
def measure_energy_and_time(func, *args, **kwargs):
    """
    Measure runtime and approximate energy usage for a given function call.
    """
    start_time = time.time()
    energy_gpu_joules = 0.0
    energy_cpu_joules = 0.0
    interval = 0.1  # Sampling interval in seconds

    def target():
        func(*args, **kwargs)

    thread = threading.Thread(target=target)
    thread.start()

    while thread.is_alive():
        # Measure GPU power using nvidia-smi
        gpu_watts = get_gpu_power()
        cpu_watts = get_cpu_power_estimate()
        time.sleep(interval)
        energy_gpu_joules += gpu_watts * interval
        energy_cpu_joules += cpu_watts * interval

    thread.join()
    end_time = time.time()
    total_time = end_time - start_time
    total_energy_joules = energy_gpu_joules + energy_cpu_joules
    total_energy_kwh = total_energy_joules * 2.77778e-7
    emissions_kg = total_energy_kwh * CARBON_INTENSITY

    return total_time, total_energy_joules, emissions_kg

# Logging functions
def log_results(task_name, phase, total_time, energy_joules, emissions_kg, filename=RESULTS_CSV):
    file_exists = os.path.isfile(filename)
    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["Task", "Phase", "Time(s)", "Energy(J)", "Emissions(kgCO2)"])
        writer.writerow([task_name, phase, total_time, energy_joules, emissions_kg])

def log_gpu_details(details, filename=GPU_DETAILS_CSV):
    file_exists = os.path.isfile(filename)
    with open(filename, 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=details[0].keys())
        if not file_exists:
            writer.writeheader()
        writer.writerows(details)

###############################
# Load Data
###############################
df = pd.read_csv('tweets_processed.csv')

# Use a subset of data for quicker tests (e.g., 10% of original data)
df = df.sample(frac=0.6, random_state=42).reset_index(drop=True)

from sklearn.model_selection import train_test_split

X = df['lemmatized_tweets']
y = df['VADER_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

###############################
# Preprocessing and Tokenization
###############################
MAX_LENGTH = math.ceil((X_train.apply(lambda x: len(str(x).split())).mean()))+2

PAD_TOKEN = "<|pad|>"
EOS_TOKEN = "<|endoftext|>"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2",
    pad_token=PAD_TOKEN,
    eos_token=EOS_TOKEN,
    max_length=MAX_LENGTH,
    is_split_into_words=True)

X_train = [str(ex) + EOS_TOKEN for ex in X_train]
X_test = [str(ex) + EOS_TOKEN for ex in X_test]

X_train_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                      truncation=True, padding="max_length", add_special_tokens=True)['input_ids'] for x in X_train]
X_test_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                     truncation=True, padding="max_length", add_special_tokens=True)['input_ids'] for x in X_test]

X_train_in = tf.squeeze(tf.convert_to_tensor(X_train_), axis=1)
X_test_in = tf.squeeze(tf.convert_to_tensor(X_test_), axis=1)

X_train_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                           truncation=True, padding="max_length", add_special_tokens=True)["attention_mask"] for x in X_train]
X_test_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                          truncation=True, padding="max_length", add_special_tokens=True)["attention_mask"] for x in X_test]

X_train_mask = tf.squeeze(tf.convert_to_tensor(X_train_mask_), axis=1)
X_test_mask = tf.squeeze(tf.convert_to_tensor(X_test_mask_), axis=1)

###############################
# Model Setup
###############################
model = TFGPT2Model.from_pretrained("gpt2", use_cache=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id)
model.training = True
model.resize_token_embeddings(len(tokenizer))

# Freeze GPT2 layers
for layer in model.layers:
    layer.trainable = False

input_ = tf.keras.layers.Input(shape=(None,), dtype='int32')
mask_ = tf.keras.layers.Input(shape=(None,), dtype='int32')
x = model(input_, attention_mask=mask_)
x = tf.reduce_mean(x.last_hidden_state, axis=1)
x = tf.keras.layers.Dense(16, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(3, activation='softmax')(x)

clf = tf.keras.Model([input_, mask_], output)

base_learning_rate = 0.0005
optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

clf.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

callbacks = tf.keras.callbacks.EarlyStopping(
    monitor="accuracy", verbose=1, patience=3, restore_best_weights=True)

def map_sentiment(value):
  if value == 'Negative':
    return 0
  if value == 'Neutral':
    return 1
  if value == 'Positive':
    return 2

y_train_ = y_train.map(map_sentiment)
y_test_ = y_test.map(map_sentiment)

y_train_in = tf.constant(y_train_, dtype=tf.int32)
y_test_in = tf.constant(y_test_, dtype=tf.int32)

tf.config.experimental_run_functions_eagerly(True)

def train_model():
    clf.fit([X_train_in, X_train_mask], y_train_in, epochs=30, batch_size=32, validation_split=0.2, callbacks=callbacks)

###############################
# Gather GPU Details
###############################
gpu_details = get_gpu_details()
print("GPU Details and Insights:")
for detail in gpu_details:
    print(detail)
log_gpu_details(gpu_details)

###############################
# Training with Energy Measurement
###############################
print("Starting training...")
train_time, train_energy_j, train_emissions_kg = measure_energy_and_time(train_model)
print(f"Training completed in {train_time:.2f}s, Energy: {train_energy_j:.2f}J, Emissions: {train_emissions_kg:.4f}kg CO2")
log_results("GPT2_TransferLearning", "Training", train_time, train_energy_j, train_emissions_kg)

###############################
# Evaluation with Energy Measurement
###############################
print("Evaluating on test set...")
eval_time, eval_energy_j, eval_emissions_kg = measure_energy_and_time(clf.evaluate, [X_test_in, X_test_mask], y_test_in)
print(f"Evaluation completed in {eval_time:.2f}s, Energy: {eval_energy_j:.2f}J, Emissions: {eval_emissions_kg:.4f}kg CO2")
log_results("GPT2_TransferLearning", "Evaluation", eval_time, eval_energy_j, eval_emissions_kg)

clf.training = False
y_pred = clf.predict([X_test_in, X_test_mask])

print("Done. Results saved to:", RESULTS_CSV)


All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


GPU Details and Insights:
{'GPU': 'Tesla T4', 'Memory Total (MB)': 15360, 'Memory Free (MB)': 1037, 'Memory Used (MB)': 14322, 'GPU Utilization (%)': 0, 'Memory Utilization (%)': 0, 'CUDA Version': 12020}
Starting training...
Epoch 1/30


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


24/24 [==============================] - ETA: 0s - loss: 2.7486 - accuracy: 0.3772

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


24/24 [==============================] - 28s 1s/step - loss: 2.7486 - accuracy: 0.3772 - val_loss: 1.1715 - val_accuracy: 0.4021
Epoch 2/30
24/24 [==============================] - 15s 641ms/step - loss: 1.3915 - accuracy: 0.4064 - val_loss: 1.0751 - val_accuracy: 0.4550
Epoch 3/30
24/24 [==============================] - 17s 707ms/step - loss: 1.1530 - accuracy: 0.4409 - val_loss: 1.0388 - val_accuracy: 0.5079
Epoch 4/30
24/24 [==============================] - 15s 627ms/step - loss: 1.0602 - accuracy: 0.4622 - val_loss: 1.0287 - val_accuracy: 0.5238
Epoch 5/30
24/24 [==============================] - 15s 616ms/step - loss: 1.0772 - accuracy: 0.4250 - val_loss: 1.0329 - val_accuracy: 0.5291
Epoch 6/30
24/24 [==============================] - 15s 633ms/step - loss: 1.0402 - accuracy: 0.4768 - val_loss: 1.0315 - val_accuracy: 0.5344
Epoch 7/30
24/24 [==============================] - 17s 727ms/step - loss: 1.0343 - accuracy: 0.4794 - val_loss: 1.0298 - val_accuracy: 0.5132
Epoch 8/30
24

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


10/10 [==============================] - 2s 220ms/step - loss: 1.0069 - accuracy: 0.5414
Evaluation completed in 18.87s, Energy: 711.08J, Emissions: 0.0001kg CO2


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


10/10 [==============================] - 2s 194ms/step
Done. Results saved to: energy_measurements_gpu.csv


In [1]:
!nvidia-smi

Tue Dec 10 13:23:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--